In [22]:
import numpy as np
import pandas as pd
import csv
from scipy.stats import kruskal,mannwhitneyu

In [2]:
def soft_max (x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [6]:
datacsv = open("../General Survey.csv")
reader = list(csv.reader(datacsv, delimiter=',', quotechar='"'))
fields = [x.strip() for x in reader.pop(0)]
print(fields)

['Timestamp', 'Please rank the following (No repeat ranks, must rank each row) [Production Budget]', 'Please rank the following (No repeat ranks, must rank each row) [Producer]', 'Please rank the following (No repeat ranks, must rank each row) [Average Critic Score (Rotten Tomatoes)]', 'Please rank the following (No repeat ranks, must rank each row) [IMDB Rating]', 'Please rank the following (No repeat ranks, must rank each row) [IMDB Votes]', 'Please rank the following (No repeat ranks, must rank each row) [Average Audience Score (Rotten Tomatoes)]', 'Please rank the following (No repeat ranks, must rank each row) [Return on Investment]', 'Please rank the following (No repeat ranks, must rank each row) [Worldwide Gross]', 'How much do you enjoy researching movies, such as browsing IMDB, Wikipedia pages, and looking up actors?', 'How much do you enjoy watching movies?', 'How often do you watch the Oscars or similar movie award shows?', 'What is your current age?', 'Are you currently at

In [7]:
reader = [[cell.split("(")[0].strip() for cell in col]for col in reader]

In [12]:
ratings = pd.DataFrame(reader).values[:,1:-5].astype(int)
ratings = ratings.T
ratings = np.delete(ratings,1,axis=0) # removing producer
trimmed_fields = [x.split("[")[1].split("]")[0].strip().split('(')[0] for x in fields[1:-5]]
trimmed_fields.pop(1) # removing producer
print(trimmed_fields)
means = [x.mean() for x in ratings]
# print(means)

softs = soft_max(means)
print("Soft Max Values")
zipped = [[f,v]for f,v in zip(trimmed_fields,softs)]
zipped.sort(key=lambda x: x[0], reverse=True)
for f,v in zipped:
    print("%s: %s" % (f,v))

['Production Budget', 'Average Critic Score ', 'IMDB Rating', 'IMDB Votes', 'Average Audience Score ', 'Return on Investment', 'Worldwide Gross']
Soft Max Values
Worldwide Gross: 0.167137582418
Return on Investment: 0.0607032238123
Production Budget: 0.0336580824017
IMDB Votes: 0.0209448803779
IMDB Rating: 0.101374068127
Average Critic Score : 0.557769349727
Average Audience Score : 0.0584128131355


In [13]:
logregcoefs = [['IMDB Rating', -0.34300168957186966], ['IMDB Votes', 1.3669196119522085], ['Average Critic Score', 0.57744046973782626], ['Average Audience Score', -0.40653727105115345], ['Production Budget', -0.27290349924072632], ['Worldwide Gross', 0.69783373108027336], ['Return on Investment', 1.3457350571892375]]
logregcoefs = np.array(logregcoefs)
logregcoefs[:,1] = soft_max(logregcoefs[:,1].astype(float))
logregcoefs = logregcoefs.tolist()
logregcoefs.sort(key=lambda x: x[0], reverse=True)
for f,v in logregcoefs:
    print("%s: %s" % (f,v))

Worldwide Gross: 0.14675816645915077
Return on Investment: 0.2805318966523604
Production Budget: 0.05559253199638893
IMDB Votes: 0.2865382361658364
IMDB Rating: 0.05182904390551388
Average Critic Score: 0.130111639147387
Average Audience Score: 0.04863848567336266


In [29]:
zipped1 = [z[1] for z in zipped] # get just the values
logreg1 = [float(l[1]) for l in logregcoefs]
# print(zipped1)
# print(logreg1)

test = kruskal(np.array(zipped1),np.array(logreg1))
test2 = mannwhitneyu(np.array(zipped1),np.array(logreg1))
print(test)


KruskalResult(statistic=0.19999999999999574, pvalue=0.65472084601858027)
